In [2]:
import Eisenia

┌ Info: Recompiling stale cache file /Users/cameronprybol/.julia/compiled/v1.2/Eisenia/cMB3l.ji for Eisenia [ff91eec8-cf20-4569-aeea-763e189e8d12]
└ @ Base loading.jl:1240


In [3]:
import BioSequences

In [4]:
import Random

In [5]:
import Dates

In [6]:
LENGTH = 4

4

In [7]:
Random.seed!(LENGTH)

Random.MersenneTwister(UInt32[0x00000004], Random.DSFMT.DSFMT_state(Int32[-1125970028, 1072950084, 1987879523, 1073737715, 1318759160, 1073082403, -665387610, 1073002484, 775432869, 1072783382  …  -1193370493, 1072918807, 503071495, 1073572056, -1375397214, 586182507, 2038247757, 798803726, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000

In [8]:
seq = BioSequences.randdnaseq(LENGTH)

4nt DNA Sequence:
GTTT

In [9]:
id = Random.randstring(LENGTH)

"QTGJ"

In [10]:
record = BioSequences.FASTA.Record(id, seq)

BioSequences.FASTA.Record:
   identifier: QTGJ
  description: <missing>
     sequence: GTTT

In [11]:
canonical_kmers =
    unique([BioSequences.canonical(kmer) for (index, kmer) in
            BioSequences.each(BioSequences.DNAKmer{3}, BioSequences.FASTA.sequence(record))])

2-element Array{BioSequences.Kmer{BioSymbols.DNA,3},1}:
 AAC
 AAA

In [12]:
observations = [record]

1-element Array{BioSequences.FASTA.Record,1}:
 BioSequences.FASTA.Record:
   identifier: QTGJ
  description: <missing>
     sequence: GTTT

In [13]:
?Eisenia.build_stranded_kmer_graph

```
build_stranded_kmer_graph(canonical_kmers, observations)
```

Create a weighted, strand-specific kmer (de bruijn) graph from a set of kmers and a series of sequence observations in FASTA format.

Each vertex is represented as an integer, 1..n That vertex integer maps to a kmer in an n-length list of kmers Each vertex is backed up by evidence in the form of (read # => (index in read => orientation))

Each edge is represented by a directional pair of (vertex => vertex)


In [22]:
stranded_kmer_graph = Eisenia.build_stranded_kmer_graph(canonical_kmers, observations)

{4, 2} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [23]:
filename = "$(BioSequences.FASTA.identifier(record)).strand_specific.svg"
Eisenia.plot_stranded_kmer_graph(stranded_kmer_graph, filename=filename)
HTML("""
<image src="$filename" width=50%>
""")

HTML{String}("<image src=\"QTGJ.strand_specific.svg\" width=50%>\n")

In [24]:
# normalize me
Eisenia.normalize!(stranded_kmer_graph)

(stranded_kmer_graph.vprops[vertex])[:coverage] = Pair{Int64,Pair{Int64,Bool}}[]
(stranded_kmer_graph.vprops[vertex])[:coverage] = Pair{Int64,Pair{Int64,Bool}}[]
(stranded_kmer_graph.vprops[vertex])[:coverage] = Pair{Int64,Pair{Int64,Bool}}[1 => (1 => 1)]
(stranded_kmer_graph.vprops[vertex])[:coverage] = Pair{Int64,Pair{Int64,Bool}}[1 => (2 => 1)]
(stranded_kmer_graph.eprops[edge])[:coverage] = Pair{Pair{Int64,Pair{Int64,Bool}},Pair{Int64,Pair{Int64,Bool}}}[]
(stranded_kmer_graph.eprops[edge])[:coverage] = Pair{Pair{Int64,Pair{Int64,Bool}},Pair{Int64,Pair{Int64,Bool}}}[(1 => (1 => 1)) => (1 => (2 => 1))]


In [25]:
import LightGraphs

In [45]:
for vertex in LightGraphs.vertices(stranded_kmer_graph)
    vertex′ = stranded_kmer_graph.gprops[:reverse_complement_map][vertex]
    
    vertex_coverage = stranded_kmer_graph.vprops[vertex][:coverage]
    vertex_coverage′ = Vector{eltype(vertex_coverage)}(undef, length(vertex_coverage))
    
    vertex′_coverage = stranded_kmer_graph.vprops[vertex′][:coverage]
    vertex′_coverage′ = Vector{eltype(vertex′_coverage)}(undef, length(vertex′_coverage))
    
    for (i, (read_index, (index_in_read, orientation))) in enumerate(vertex_coverage)
        vertex_coverage′[i] = (read_index => (index_in_read => !orientation))
    end
    for (i, (read_index, (index_in_read, orientation))) in enumerate(vertex′_coverage)
        vertex′_coverage′[i] = (read_index => (index_in_read => !orientation))
    end
    
    unique!(append!(stranded_kmer_graph.vprops[vertex][:coverage], vertex′_coverage′))
    unique!(append!(stranded_kmer_graph.vprops[vertex′][:coverage], vertex_coverage′))
end

In [46]:
for vertex in LightGraphs.vertices(stranded_kmer_graph)
    @show stranded_kmer_graph.vprops[vertex][:coverage]
end

(stranded_kmer_graph.vprops[vertex])[:coverage] = Pair{Int64,Pair{Int64,Bool}}[1 => (2 => 0)]
(stranded_kmer_graph.vprops[vertex])[:coverage] = Pair{Int64,Pair{Int64,Bool}}[1 => (1 => 0)]
(stranded_kmer_graph.vprops[vertex])[:coverage] = Pair{Int64,Pair{Int64,Bool}}[1 => (1 => 1)]
(stranded_kmer_graph.vprops[vertex])[:coverage] = Pair{Int64,Pair{Int64,Bool}}[1 => (2 => 1)]


In [47]:
filename = "$(BioSequences.FASTA.identifier(record)).strand_specific.svg"
Eisenia.plot_stranded_kmer_graph(stranded_kmer_graph, filename=filename)
HTML("""
<image src="$filename" width=50%>
""")

HTML{String}("<image src=\"QTGJ.strand_specific.svg\" width=50%>\n")

In [21]:
for edge in LightGraphs.edges(stranded_kmer_graph)
    @show edge
    @show stranded_kmer_graph.eprops[edge][:coverage]
    
end

edge = Edge 1 => 2
(stranded_kmer_graph.eprops[edge])[:coverage] = Pair{Pair{Int64,Pair{Int64,Bool}},Pair{Int64,Pair{Int64,Bool}}}[]
edge = Edge 3 => 4
(stranded_kmer_graph.eprops[edge])[:coverage] = Pair{Pair{Int64,Pair{Int64,Bool}},Pair{Int64,Pair{Int64,Bool}}}[(1 => (1 => 1)) => (1 => (2 => 1))]


In [ ]:
# replot with normalized

In [ ]:
# make sure that each read has the same color (did I already do that?)